In [2]:
EMPTY = "L"
OCCUPIED = "#"
FLOOR = "."

def setup_seats(lines):
    seats = {}
    for row, line in enumerate(lines, start=0):
        seats_row = {}
        for column, char in enumerate(line, start=0):
            seats_row[column] = char
        seats[row] = seats_row
    return seats

def iterate_seats(seats):
    result = {}
    for row in seats:
        result[row] = {}
        for column in seats.get(row):
            place = seats.get(row).get(column)
            adjacents = get_adjacent_seats(seats, row, column)
            result[row][column] = set_seat(place, adjacents)
    return result

def set_seat(place, adjacents):
    return OCCUPIED if place == EMPTY and OCCUPIED not in adjacents else EMPTY if place == OCCUPIED and adjacents.count(OCCUPIED) > 3 else place

def get_adjacent_seats(seats, row, column):
    adjacent_seats = {
        row - 1: {column - 1, column, column + 1},
        row: {column -1, column + 1},
        row + 1: {column - 1, column, column + 1}
    }
    result = []
    for row in adjacent_seats:
        if seats.get(row):
            for column in adjacent_seats.get(row):
                seat = seats.get(row).get(column)
                if seat:
                    result.append(seat)
    return result

def count_seats(seats, kind):
    return sum(list(seats.get(row).values()).count(kind) for row in seats)
        
with open("puzzle.txt", "r") as file:
    lines = file.read().split('\n')

old_seats = setup_seats(lines)
new_seats = iterate_seats(old_seats)
while old_seats != new_seats:
    old_seats = new_seats
    new_seats = iterate_seats(old_seats)

result = count_seats(new_seats, OCCUPIED)
print(result)



2281


In [4]:
import copy

class Seat:

    def __init__(self, seat_kind):
        self.kind = seat_kind
        self.adjacents = {
            (-1, -1): None, (-1, 0): None, (-1, 1): None, 
            (0, -1): None, (0, 1): None, 
            (1, -1): None, (1, 0): None, (1, 1): None
            }

    def __eq__(self, other): 
        if not isinstance(other, Seat):
            # don't attempt to compare against unrelated types
            return NotImplemented
        return self.kind == other.kind


def setup_seats(lines):
    seats = {}
    for row, line in enumerate(lines, start=0):
        seats_row = {}
        for column, char in enumerate(line, start=0):
            seats_row[column] = Seat(char)
        seats[row] = seats_row
    return seats

def set_kind(seat):
    adjacents = list(seat.adjacents.values())
    return OCCUPIED if seat.kind == EMPTY and OCCUPIED not in adjacents else EMPTY if seat.kind == OCCUPIED and adjacents.count(OCCUPIED) > 4 else seat.kind

def iterate_seats(seats):
    for row in range(len(seats)):
        for column in range(len(seats.get(row))):
            set_adjacents(seats, row, column)

    for row in range(len(seats)):
        for column in range(len(seats.get(row))):
            seats[row][column].kind = set_kind(seats.get(row).get(column))

def set_adjacents(seats, row, column):
    if seats.get(row) and seats.get(row).get(column):
        # above adjacents
        set_adjacent(seats, row, column, -1, -1)
        set_adjacent(seats, row, column, -1, 0)
        set_adjacent(seats, row, column, -1, 1)
        
        # same row adjacents
        set_adjacent(seats, row, column, 0, -1)
        set_adjacent(seats, row, column, 0, 1)

        # below adjacents
        set_adjacent(seats, row, column, 1, -1)
        set_adjacent(seats, row, column, 1, 0)
        set_adjacent(seats, row, column, 1, 1) 


def set_adjacent(seats, row, column, row_offset, column_offset):
    if not seats.get(row).get(column).adjacents[(row_offset, column_offset)]:

        seats_to_be_updated = []
        existing_adjacent = True
        next_row = row
        next_column = column
        seat_found = None

        while existing_adjacent and not seat_found:
            seats_to_be_updated.append((next_row, next_column))
            next_row += row_offset
            next_column += column_offset

            if not seats.get(next_row):
                existing_adjacent = False
            else:
                adjacent = seats.get(next_row).get(next_column)
                if not adjacent:
                    existing_adjacent = False
                elif adjacent.kind != FLOOR:
                    seat_found = adjacent.kind
                else:
                    seat_found = adjacent.adjacents[(row_offset, column_offset)]

        if seat_found:
            for row, column in seats_to_be_updated:
                seats[row][column].adjacents[(row_offset, column_offset)] = seat_found


def create_new_seats(seats):
    new_seats = {}
    for i in range(len(seats)):
        new_seats[i] = {}
        for j in range(len(seats.get(i))):
            new_seats[i][j] = Seat(seats.get(i).get(j).kind)
    return new_seats

def count_seats(seats, kind):
    return sum([seat.kind for seat in seats.get(row).values()].count(kind) for row in seats)

with open("puzzle.txt", "r") as file:
    lines = file.read().split('\n')

old_seats = {}
new_seats = setup_seats(lines)

while old_seats != new_seats:
    old_seats = copy.deepcopy(new_seats)
    iterate_seats(new_seats) 
    new_seats = create_new_seats(new_seats)

result = count_seats(new_seats, OCCUPIED)
print(result)

rows:  97 columns:  91
2085
